In [13]:
%pip install --upgrade pymilvus[model] langchain-milvus langchain-openai
%pip install -qU langchain_community pypdf

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [14]:
from langchain_ollama import OllamaLLM

In [15]:
CONNECTION_URI = "http://localhost:19530"

In [16]:
ollama_embedding_func = OllamaLLM(model="moondream")

In [17]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Ollama Setup
def setup_ollama_pdf_vectorstore(file_path, embedding_model="moondream"):
    try:
        # 1. Load PDF
        loader = PyMuPDFLoader(file_path)
        docs = loader.load()

        # 2. Text Splitting (Optional but recommended)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )
        splits = text_splitter.split_documents(docs)

        # 3. Ollama Embeddings
        embeddings = OllamaEmbeddings(
            model=embedding_model,  # You can change this to other Ollama models
            num_gpu=6  # Optional: specify GPU usage
        )

        # 4. Create Chroma Vector Store
        vector_store = Chroma.from_documents(
            documents=splits,
            embedding=embeddings,
            collection_name="ollama_pdf_collection"
        )

        return vector_store

    except Exception as e:
        print(f"Error processing PDF: {e}")
        return None

files = os.listdir("papers")
print(files)

vector_store = setup_ollama_pdf_vectorstore(f"papers/{files[0]}")
print(f"Vector store for {files[0]} created successfully")
print("")

['2303.05510v1.pdf']
Vector store for 2303.05510v1.pdf created successfully



In [18]:
# Perform Similarity Search
contexts = ""
query = input("Enter your query: ")

if vector_store:
    results = vector_store.similarity_search(query, k=10)
    
    for doc in results:
        print("\n--- Relevant Document Excerpt ---")
        print(doc.page_content)
        contexts = contexts + doc.page_content


--- Relevant Document Excerpt ---
Published as a conference paper at ICLR 2023
Problem Statement
You are given an array a consisting of n integer numbers.
Let instability of the array be the following value:
n
max
i=1 ai −
n
min
i=1 ai.
You have to remove exactly one element from this array to minimize instability of the resulting
(n −1)-elements array. Your task is to calculate the minimum possible instability.
Input
The ﬁrst line of the input contains one integer n (2 ≤n ≤105) — the number of elements in the
array a.
The second line of the input contains n integers a1, a2, . . . , an (1 ≤ai ≤105) — elements of the
array a.
Output
Print one integer — the minimum possible instability of the array if you have to remove exactly one
element from the array a.
Examples
Input
4
1 3 3 7
Output
2
Input
2
1 100000
Output
0
Note
In the ﬁrst example you can remove 7 then instability of the remaining array will be 3 −1 = 2.
In the second example you can remove either 1 or 100000 then instability 

In [19]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage

# Initialize Ollama Chat Model
chat_model = ChatOllama(
    model="llama3.2:1b",  # You can change to other models like mistral, phi3
    temperature=0.7,  # Creativity level
    num_gpu=1  # Optional GPU configuration
)

# Simple Chat Interaction
messages = [
    SystemMessage(content="You are a helpful assistant for my research."),
    HumanMessage(content= f"Query + {query} with the context + {contexts}")
]

# Generate Response
response = chat_model.invoke(messages)
print("--- Ollama Response ---")
print(response.content)

--- Ollama Response ---
The text appears to be an abstract or introduction to a research paper discussing two new approaches for code generation using large language models. Here's a breakdown of the main points:

**Background**

* Existing code generation pipelines often use beam search or sampling algorithms during decoding, but they may not always generate correct outputs.
* Conventional transformer decoding algorithms may not be suitable for code generation.

**Proposed Approach: Planning-Guided Transformer Decoding (PG-TD)**

* PG-TD uses a planning algorithm to do lookahead search and guide the transformer decoder to generate better programs.
* The algorithm is designed specifically for code generation tasks, taking into account the problem description in a block comment.
* Two variants of PG-TD are proposed: PG-TD with cached sequences (PG-TD-Cached) and PG-TD without sequence caching (PG-TD-NoCache).

**Evaluation**

* A dataset of 165 problems from CodeContests is used to eval